In [ ]:
from jhora.utils import julian_day_number
from jhora.horoscope.chart.charts import rasi_chart, navamsa_chart
import jhora.const as const
import jhora.panchanga.drik as drik
from jhora.panchanga.drik import Place
from vedicastro.VedicAstro import VedicHoroscopeData
from timezonefinder import TimezoneFinder
from datetime import datetime
import pytz
import pandas as pd
import numpy as np

# === Основные объекты ===

planet_map = { ... }
sign_map = { ... }
nakshatra_map = { ... }
main_grahas = [ ... ]
rasi_order = [ ... ]
sign_lords = { ... }
sign_order = { ... }
D1_TO_D9_MAP = { ... }

# === Вспомогательные функции ===

def get_birth_utc_offset_str(...): ...
def translate_chart_to_russian(...): ...
def filter_main_grahas(...): ...
def add_house_info(...): ...
def decimal_to_dms(...): ...

# === Класс обёртка ===

class AstroChart:
    def __init__(self, birth_date: str, birth_time: str, latitude: str, longitude: str):
        self.birth_date = birth_date
        self.birth_time = birth_time
        self.latitude = latitude
        self.longitude = longitude
        self.dt = datetime.strptime(f"{birth_date} {birth_time}", "%d.%m.%Y %H:%M:%S")
        self.utc_offset = get_birth_utc_offset_str(self.dt, float(latitude), float(longitude))

    def build_chart(self):
        vhd = VedicHoroscopeData(
            year=self.dt.year, month=self.dt.month, day=self.dt.day,
            hour=self.dt.hour, minute=self.dt.minute, second=self.dt.second,
            utc=self.utc_offset, latitude=float(self.latitude), longitude=float(self.longitude),
            ayanamsa="Lahiri", house_system="Placidus"
        )
        vhd.get_ayanamsa()
        chart = vhd.generate_chart()
        planets_data = vhd.get_planets_data_from_chart(chart)
        df = pd.DataFrame(planets_data)

        possible_sign_cols = ["Sign", "Rasi", "SignName"]
        sign_col = next((col for col in possible_sign_cols if col in df.columns), None)
        if sign_col is None:
            raise ValueError("Не найдена колонка с названием знака зодиака (Sign/Rasi/SignName)")

        columns_mapping = {
            "Object": "Планета",
            sign_col: "Знак",
            "Retrograde": "Ретроградность",
            "DMS": "Градусы",
            "Nakshatra": "Накшатра",
            "RasiLord": "Управитель",
            "NakshatraLord": "Управитель накшатры"
        }

        df = df.rename(columns=columns_mapping)
        columns_to_delete = ['SubLord', 'SubSubLord', 'HouseNr']
        df = df.drop(columns=[col for col in columns_to_delete if col in df.columns])
        df = translate_chart_to_russian(df)
        return df

    def update_chart(self, df, filter=True):
        for col in ['LonDecDeg', 'SignLonDecDeg', 'LatDMS', 'SubLord', 'SubSubLord', 'HouseNr']:
            if col in df.columns:
                df = df.drop(columns=col)
        df = add_house_info(df)
        if filter:
            df = filter_main_grahas(df)
        return df
